In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading JSON data
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)
    
with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

### Quantize the features by k-means clustering 

In [15]:
import numpy as np

loc_list = []
length_list = []
numPts_list = []
speed_list = []

ref_loc_list = []
ref_length_list = []
ref_numPts_list = []
ref_speed_list = []
for key in jam_ref_data.keys():
    ref_loc_list.append(jam_ref_data[key]['longitude_latitude'])
    loc_list.append(jam_ref_data[key]['longitude_latitude'])
    
    ref_length_list.append(jam_ref_data[key]['length'])
    length_list.append(jam_ref_data[key]['length'])
    
    ref_numPts_list.append(float(jam_ref_data[key]['numPts']))
    numPts_list.append(float(jam_ref_data[key]['numPts']))
    
    ref_speed_list.append(jam_ref_data[key]['speed'])
    speed_list.append(jam_ref_data[key]['speed'])


test_loc_list = []
test_length_list = []
test_numPts_list = []
test_speed_list = []
for key in jam_test_data.keys():
    test_loc_list.append(jam_test_data[key]['longitude_latitude'])
    loc_list.append(jam_test_data[key]['longitude_latitude'])
    
    test_length_list.append(jam_test_data[key]['length'])
    length_list.append(jam_test_data[key]['length'])
    
    test_numPts_list.append(float(jam_test_data[key]['numPts']))
    numPts_list.append(float(jam_test_data[key]['numPts']))
    
    test_speed_list.append(jam_test_data[key]['speed'])
    speed_list.append(jam_test_data[key]['speed'])

In [16]:
loc_data = loc_list
length_data = np.array(length_list)
numPts_data = np.array(numPts_list)
speed_data = np.array(speed_list)

In [17]:
len(loc_data), len(ref_loc_list), len(test_loc_list)

(3201, 1518, 1683)

### Quantization levels: loc_k = 3, length_k = 2, numPts_k = 2, speed_k = 2

In [9]:
loc_data = [list(np.reshape(loc_data[i], np.size(loc_data[i]))) for i in range(len(loc_data))]

In [ ]:
from scipy.cluster.vq import kmeans2
from ClusterAlg import DF, KMedians

loc_k = 3
loc_label, loc_centroid, distToClusterCenter = KMedians(loc_data, loc_k, 5, DF)

In [18]:
length_k = 2
length_centroid, length_label = kmeans2(length_data, length_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

numPts_k = 2
numPts_centroid, numPts_label = kmeans2(numPts_data, numPts_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

speed_k = 2
speed_centroid, speed_label = kmeans2(speed_data, speed_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

In [13]:
distToClusterCenter, loc_label, loc_centroid

([1558.4133065759638,
  359.6847536707892,
  676.8052008225002,
  896.1548457488644,
  210.75245969919305,
  403.61087104713135,
  282.9470045187422,
  499.1815569150437,
  913.7881190794203,
  1127.4935015655594,
  1182.7626925186544,
  1557.4721800281825,
  1453.6127231083879,
  285.4708422856134,
  1478.1285050031834,
  969.8382292593757,
  1453.6127231083879,
  285.4708422856134,
  359.6847536707892,
  236.7972779918027,
  896.1548457488644,
  707.2505729364539,
  1032.027140383674,
  508.23846458954245,
  246.0106665188754,
  1098.3068856534755,
  337.41649060131635,
  96.14419132514523,
  228.61986937963306,
  223.1299749132319,
  839.449219135157,
  113.35559607420714,
  417.3539210985009,
  226.20570452014482,
  456.3647625229235,
  299.11772681896537,
  1801.302576251779,
  113.35559607420714,
  91.32592842195001,
  113.35559607420714,
  91.32592842195001,
  727.1137581563806,
  113.32756833474116,
  819.5495080103938,
  745.7199412021088,
  295.82994895950617,
  1216.48720697

In [19]:
ref_loc_label = loc_label[0:np.size(np.array(ref_loc_list),0)]
test_loc_label = loc_label[np.size(np.array(ref_loc_list),0):np.size(loc_data,0)]

ref_length_label = length_label[0:np.size(np.array(ref_length_list),0)]
test_length_label = length_label[np.size(np.array(ref_length_list),0):np.size(length_data,0)]

ref_numPts_label = numPts_label[0:np.size(np.array(ref_numPts_list),0)]
test_numPts_label = numPts_label[np.size(np.array(ref_numPts_list),0):np.size(numPts_data,0)]

ref_speed_label = speed_label[0:np.size(np.array(ref_speed_list),0)]
test_speed_label = speed_label[np.size(np.array(ref_speed_list),0):np.size(speed_data,0)]

In [20]:
len(ref_loc_label), len(test_loc_label), len(ref_length_label), len(test_length_label), \
len(ref_numPts_label), len(test_numPts_label), len(ref_speed_label), len(test_speed_label)

(1518, 1683, 1518, 1683, 1518, 1683, 1518, 1683)

In [24]:
quantized_jam_ref_data = {}

i = 0
for key in jam_ref_data.keys():
    data = {'uuid': jam_ref_data[key]['uuid'],
            'longitude_latitude': jam_ref_data[key]['longitude_latitude'],
            'length': jam_ref_data[key]['length'],
            'numPts': float(jam_ref_data[key]['numPts']),
            'speed': jam_ref_data[key]['speed'],
            'startTime': jam_ref_data[key]['startTime'],
            'loc_label': float(ref_loc_label[i]),
            'length_label': float(ref_length_label[i]),
            'numPts_label': float(ref_numPts_label[i]),
            'speed_label': float(ref_speed_label[i])}
    quantized_jam_ref_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_ref_data.json', 'w') as json_file:
    json.dump(quantized_jam_ref_data, json_file)

In [25]:
quantized_jam_test_data = {}

i = 0
for key in jam_test_data.keys():
    data = {'uuid': jam_test_data[key]['uuid'],
            'longitude_latitude': jam_test_data[key]['longitude_latitude'],
            'length': jam_test_data[key]['length'],
            'numPts': float(jam_test_data[key]['numPts']),
            'speed': jam_test_data[key]['speed'],
            'startTime': jam_test_data[key]['startTime'],
            'loc_label': float(test_loc_label[i]),
            'length_label': float(test_length_label[i]),
            'numPts_label': float(test_numPts_label[i]),
            'speed_label': float(test_speed_label[i])}
    quantized_jam_test_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_test_data.json', 'w') as json_file:
    json.dump(quantized_jam_test_data, json_file)